In [65]:
import pandas as pd 
import librosa
import os
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import glob
from sklearn.ensemble import RandomForestClassifier


In [88]:
meta = pd.read_csv('meta/esc50.csv')
file = meta.iloc[0]['filename']
base_folder = 'audio'
csv_folder = 'csvs'

In [5]:
search = os.path.join(csv_folder, "*.wav")
all_csv = glob.glob(search)
print(len(all_csv), all_csv[0])
arr = loadtxt(all_csv[0])



2000 csvs/1-100032-A-0.wav


array([ 5.94373866e-03,  1.39346351e-03,  3.11433312e-01,  1.06288669e-02,
        1.05889588e-02,  3.51528704e-02,  2.24070629e-03,  8.62068966e-03,
       -4.84470221e+01, -2.34449032e-01, -2.04631568e-01,  5.53270057e-02,
        4.57109316e-02,  3.71470441e-02, -8.13664681e-02,  1.36792224e-02,
        3.13885758e-02, -1.34165090e-02, -1.90663827e-02,  2.57283723e-02,
       -3.39445994e-02,  3.05371574e-04,  4.25360391e-04,  1.05622676e-03,
        9.52823188e-04,  1.50297907e-01,  5.31016596e-04,  6.16056748e-05,
        2.88572252e-05,  1.42550472e-04,  1.06223578e-04,  1.91021227e-03,
        5.28212285e-03,  4.29684865e-02])

In [41]:
# how to decide which to take to train and which to test?
from sklearn.model_selection import train_test_split
X = sorted_main['filename']
y = sorted_main['category']
columns = y.unique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)
train.groupby('category').count()
test.groupby('category').count()

,filename
category,
airplane,10
breathing,10
brushing_teeth,10
can_opening,10
car_horn,10
cat,10
chainsaw,10
chirping_birds,10
church_bells,10


In [55]:
# for each file in training we need to get it's features 
X_train_vectors = []
for index, filename in X_train.items():
    csv_file_path = os.path.join(csv_folder, filename)
    features = loadtxt(csv_file_path, delimiter=',')
    X_train_vectors.append(features)

print(len(X_train))
X_test_vectors = []
for index, filename in X_test.items():
    csv_file_path = os.path.join(csv_folder, filename)
    features = loadtxt(csv_file_path, delimiter=',')
    X_test_vectors.append(features)

print(len(X_test_vectors))

1500
500


In [56]:
# we can use feature scaling, but later 
classifier = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=0)
classifier.fit(X_train_vectors, y_train)
y_pred = classifier.predict(X_test_vectors)

ValueError: could not convert string to float: 'clapping'

In [58]:
# add label encoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)
y_train_encoded

array([ 9, 24, 11, ..., 16,  0, 12])

In [83]:
# we can use feature scaling, but later 
classifier = RandomForestClassifier(n_estimators=500, max_depth=20, random_state=0)
classifier.fit(X_train_vectors, y_train_encoded)
y_pred = classifier.predict(X_test_vectors)

In [85]:

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_encoded, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_encoded, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_encoded, y_pred)))


Mean Absolute Error: 8.992
Mean Squared Error: 227.336
Root Mean Squared Error: 15.077665601809851


In [73]:

print(y_pred[0])
print(classification_report(y_test_encoded, y_pred, target_names=test.category.unique()))

18
                  precision    recall  f1-score   support

             dog       0.40      0.40      0.40        10
 keyboard_typing       1.00      0.30      0.46        10
  chirping_birds       0.56      0.50      0.53        10
     mouse_click       0.56      0.50      0.53        10
         insects       0.83      0.50      0.62        10
        clapping       1.00      0.30      0.46        10
     can_opening       0.57      0.40      0.47        10
       fireworks       0.43      0.30      0.35        10
    toilet_flush       0.60      0.60      0.60        10
             cat       0.75      0.60      0.67        10
            crow       0.67      0.80      0.73        10
        sneezing       0.00      0.00      0.00        10
      helicopter       0.10      0.10      0.10        10
        car_horn       0.57      0.40      0.47        10
     clock_alarm       0.50      0.80      0.62        10
   pouring_water       0.50      0.40      0.44        10
      cloc

In [86]:
# we can use feature scaling, but later 
regressor = RandomForestClassifier(n_estimators=500, max_depth=60, random_state=0)
regressor.fit(X_train_vectors, y_train_encoded)
y_pred = regressor.predict(X_test_vectors)

In [87]:

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_encoded, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_encoded, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_encoded, y_pred)))


Mean Absolute Error: 8.488
Mean Squared Error: 215.584
Root Mean Squared Error: 14.682779028508193


In [ ]:
# BEST CLASSES: wind, vacuum cleaner, church bells, door wood creaks, crow, thunderstorm, hand saw, engine, cat, clock alarm


In [ ]:
# do cross validation 